In [20]:
import pandas as pd
import string

def split_x(x,n=20):
    results = []
    
    if '+1)' in x:
        for i in range(1,n):
            text = x[x.find(f'+{i})'):x.find(f'+{i+1})')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '1)' in x:
        for i in range(1,n):
            text = x[x.find(f'{i})'):x.find(f'{i+1})')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '1.' in x:
        for i in range(1,n):
            text = x[x.find(f'{i}.'):x.find(f'{i+1}.')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '+' in x:
        for text in x.split('+'):
            if len(text) > 5:
                results.append('+'+str(text))
    
    elif '(A)' in x:
        for i,j in zip(string.ascii_uppercase,string.ascii_uppercase[1:]+'A'):
            text = x[x.find(f'({i})'):x.find(f'({j})')]
            if len(text) > 5:
                results.append(str(text))
            
    return results

In [21]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)[['LCNO','46A','47A']]
idx = df.sample(20).index
df.loc[idx]

,LCNO,46A,47A
20079,265B20LC008636,1. SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL QUO...,1. FIVE PERCENT MORE OR LESS IN QUANTITY ON EA...
617,403420LC21000096,1.SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AND...,1.FEE OF USD50.00 OR EQUIVALENT AND RELEVANT C...
4956,LC44289C100770,+SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS INDI...,+DOCUMENTS ISSUED EARLIER THAN L/C ISSUING DAT...
3954,1NV2/00062/1153,+ SIGNED COMMERCIAL INVOICE IN 2 ORIGINALS\r\n...,+ L/C NUMBER OR INVOICE NUMBER MUST NOT INDICA...
481,M07BU2103SS00025,+SIGNED COMMERCIAL INVOICE IN 2 FOLD\r\n+SIGNE...,+/- 5 PCT IN QUANTITY AND AMOUNT ARE ACCEPTABL...
1225,00001010329477,+MANUALLY SIGNED COMMERCIAL INVOICE IN 2 COPY(...,+BENEFICIARY'S TEL: 886-2-27122211.\r\n+10 PCT...
3526,NHOU21S00026-068,+ SIGNED COMMERCIAL INVOICE IN 1 ORIGINAL INDI...,+ CHARTER PARTY B/L IS ACCEPTABLE.\r\n+ THIRD ...
7923,DC MGK179487,+SIGNED COMMERCIAL INVOICE IN DUPLICATE.\r\n+P...,+BILLS OF LADING MUST SPECIFY THE NAME OF LOAD...
1753,710LCS-67767467,+ COMMERCIAL INVOICE IN 3 ORIGINAL\r\n+ FULL S...,+ T.T.REIMBURSEMENT : PROHIBITED\r\n+ QUANTITY...
390,FAHGLCST210042IS,+ FULL SET (INCLUDING 3 ORIGINALS AND 2 NON-NE...,+ ALL DOCUMENTS MUST BE ISSUED IN ENGLISH.\r\n...


In [22]:
押匯文件清單 = pd.read_excel('../data/ALL-LC 押匯文件清單.xlsx')
押匯文件清單.head()

,LC NO,銀行代號,條款,匯票_正本,匯票_副本,商業發票_正本,商業發票_副本,裝箱單_正本,裝箱單_副本,保險_正本,...,產地商會證明_正本,產地商會證明_副本,檢驗證明_正本,檢驗證明_副本,重量證明_正本,重量證明_副本,船證_正本,船證_副本,受益人_正本,受益人_副本
0,ZU20ILC04142,9,1. BENEFICIARY'S SIGNED COMMERCIAL INVOICE FOR...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,YL200-14098K,21,SIGNED COMMERCIAL INVOICE IN 2 ORIGINALS AND 1...,NaN,NaN,2.0,1.0,2.0,1.0,2.0,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1/3,NaN
2,TF2028957357RYD,4,1.FULL SET OF CLEAN ON BOARD OCEAN BILL OF LAD...,NaN,NaN,3.0,NaN,3.0,NaN,2.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F0NN2/00278/2605,7,SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS\nAND...,NaN,NaN,3.0,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LCC0110202002398,8,1.SIGNED COMMERCIAL INVOICE IN 5 ORIGINALS IND...,NaN,NaN,5.0,NaN,3.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


In [23]:
寶典 = ['BENEFICIARYS DRAFT',
'BILL OF LADING','TRANSPORT DOCUMENTS','B/L','MARINE/OCEAN BILL OF LADING','MARINE BILL OF LADING','OCEAN BILL OF LADING',
'COMMERCIAL INVOICE','INVOICE','INV.',
'PACKING LIST','PACKING','P/L',
'INSURANCE POLICY/CERTIFICATE','I/P',
'MARINE INSURANCE POLICY' ,'MARINE CARGO POLICY','CARGO POLICY','INSURANCE AGENCY',
'CONSULAR DECLARATION','CONSULAR INVOICE','CONSULAGE','CONSULAR LEGALIZED INVOICE',
'CERTIFICATE OF ORIGIN','COO',
'TEST CERTIFICAT/INSPECTION','SURVEY REPORT','CERTIFICATE OF QUALITY','TEST CERTIFICAT','TEST INSPECTION'
'CERTIFICATE OF QUANTITY','COQ','WEIGHT MEMO',
'SHIPPING COMPANY CERTIFICATE','CERTIFICATE FROM SHIPPING COMPANY',
'FUMIGATION CERTIFICATE',
'BENEFICIARYS CERTIFICATE',
'CERTIFICATE OF ANALYSIS','COA']

In [24]:
x1 = []
x2 = []

for idx in df.index:
    lcno = df.loc[idx,'LCNO']
    x1 = x1 + [ lcno + ' ' + i for i in split_x(str(df.loc[idx,'46A']))]
    x2 = x2 + [ lcno + ' ' + i for i in split_x(str(df.loc[idx,'47A']))]
len(x1),len(x2)

(23007, 36592)

In [25]:
from tqdm import tqdm_notebook as tqdm
def Collection_methodV2(x1,x2,寶典):
    labels = {}
    
    # 46A
    for i,j in tqdm(zip(x1,x2)):
        products = []
        for p in 寶典:
            if p in i:
                products.append(p+' '+str('46A'))
            #if p in j:
            #    products.append(p+' '+str('47A'))
        try:
            product = max(products,key=len)
            if product[:-4] in i:
                labels[i] = product
            #if product[:-4] in j:
            #    labels[j] = product
        except:
            labels[i] = 'not find 46A'
    
    # 47A
    for i,j in tqdm(zip(x1,x2)):
        products = []
        for p in 寶典:
            #if p in i:
            #    products.append(p+' '+str('46A'))
            if p in j:
                products.append(p+' '+str('47A'))
        try:
            product = max(products,key=len)
            #if product[:-4] in i:
            #    labels[i] = product
            if product[:-4] in j:
                labels[j] = product
        except:
            labels[j] = 'not find 47A'
    
    predict = pd.DataFrame(index=labels.keys(),columns=['predict'])
    predict['predict'] = labels.values()
    predict = predict.reset_index()
    predict.columns = ['string_X_train','predict']
    return predict

In [26]:
result = Collection_methodV2(x1,x2,寶典)

<ipython-input-25-db5aa0ac1af0>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i,j in tqdm(zip(x1,x2)):


<ipython-input-25-db5aa0ac1af0>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i,j in tqdm(zip(x1,x2)):


In [27]:
result

,string_X_train,predict
0,6281MLC00000321 1. SIGNED COMMERCIAL INVOICE I...,COMMERCIAL INVOICE 46A
1,6281MLC00000321 2. FULL SET OF SIGNED 'CLEAN O...,CERTIFICATE FROM SHIPPING COMPANY 46A
2,6281MLC00000321 3.MARINE INSURANCE POLICY OR C...,MARINE INSURANCE POLICY 46A
3,6281MLC00000321 4. PACKING LIST IN 3 ORIGINALS...,PACKING LIST 46A
4,6281MLC00000321 5. CERTIFICATE OF ORIGIN ISSUE...,CERTIFICATE OF ORIGIN 46A
...,...,...
33390,0505621IM0000131 2. DRAFT DRAWN UNDER THIS CRE...,not find 47A
33391,0505621IM0000131 3. UNLESS OTHERWISE EXPRESSLY...,not find 47A
33392,0505621IM0000131 4. PRESENTATION TO BE MADE WI...,not find 47A
33393,0505621IM0000131 5. IN ACCORDANCE WITH THE PRO...,not find 47A


In [28]:
result[result['predict']!= 'not find'].shape

(33395, 2)

In [29]:
result[result['predict'].str.contains('not find')].shape

(17042, 2)

In [30]:
result

,string_X_train,predict
0,6281MLC00000321 1. SIGNED COMMERCIAL INVOICE I...,COMMERCIAL INVOICE 46A
1,6281MLC00000321 2. FULL SET OF SIGNED 'CLEAN O...,CERTIFICATE FROM SHIPPING COMPANY 46A
2,6281MLC00000321 3.MARINE INSURANCE POLICY OR C...,MARINE INSURANCE POLICY 46A
3,6281MLC00000321 4. PACKING LIST IN 3 ORIGINALS...,PACKING LIST 46A
4,6281MLC00000321 5. CERTIFICATE OF ORIGIN ISSUE...,CERTIFICATE OF ORIGIN 46A
...,...,...
33390,0505621IM0000131 2. DRAFT DRAWN UNDER THIS CRE...,not find 47A
33391,0505621IM0000131 3. UNLESS OTHERWISE EXPRESSLY...,not find 47A
33392,0505621IM0000131 4. PRESENTATION TO BE MADE WI...,not find 47A
33393,0505621IM0000131 5. IN ACCORDANCE WITH THE PRO...,not find 47A


In [31]:
p2c = {'BENEFICIARYS DRAFT':'匯票',

'BILL OF LADING':'提單',
'TRANSPORT DOCUMENTS':'提單',
'B/L':'提單',
'MARINE/OCEAN BILL OF LADING':'提單',
'MARINE BILL OF LADING':'提單',
'OCEAN BILL OF LADING':'提單',

'COMMERCIAL INVOICE':'商業發票',
'INVOICE':'商業發票',
'INV.':'商業發票',

'PACKING LIST':'裝箱單',
'PACKING':'裝箱單',
'P/L':'裝箱單',

'INSURANCE POLICY/CERTIFICATE':'保險單據',
'INSURANCE POLICY':'保險單據',
'I/P':'保險單據',
'MARINE INSURANCE POLICY':'保險單據',
'MARINE CARGO POLICY':'保險單據',
'CARGO POLICY':'保險單據',
'INSURANCE AGENCY':'保險單據',

'CONSULAR DECLARATION':'領事簽證',
'CONSULAR INVOICE':'領事簽證',
'CONSULAGE':'領事簽證',
'CONSULAR LEGALIZED INVOICE':'領事簽證',

'CERTIFICATE OF ORIGIN':'產地(商會)證明',
'COO':'產地(商會)證明',

'TEST CERTIFICAT':'檢驗證明',
'TEST INSPECTION':'檢驗證明',
'SURVEY REPORT CERTIFICATE OF QUALITY':'檢驗證明',
'CERTIFICATE OF QUANTITY':'檢驗證明',
'CERTIFICATE OF ANALYSIS':'檢驗證明',
'COA':'檢驗證明',

'COQ':'重量證明',
'WEIGHT MEMO':'重量證明',

'SHIPPING COMPANY CERTIFICATE':'船證',
'CERTIFICATE FROM SHIPPING COMPANY':'船證',

'FUMIGATION CERTIFICATE':'薰蒸證明書',

'BENEFICIARYS CERTIFICATE':'受益人證明',

}

In [32]:
def f(x):
    if x != 'not find':
        return x[-3:]
    else:
        return x
def g(x):
    if x != 'not find':
        return x[:-4]
    else:
        return x

def k(x):
    return x.split(' ')[0]

def j(x):
    return ' '.join(x.split(' ')[1:])

result['from'] = result['predict'].apply(f)
result['predict'] = result['predict'].apply(g)
result['LCNO'] = result['string_X_train'].apply(k)
result['string_X_train'] = result['string_X_train'].apply(j)
result

,string_X_train,predict,from,LCNO
0,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,COMMERCIAL INVOICE,46A,6281MLC00000321
1,2. FULL SET OF SIGNED 'CLEAN ON BOARD' OCEAN B...,CERTIFICATE FROM SHIPPING COMPANY,46A,6281MLC00000321
2,3.MARINE INSURANCE POLICY OR CERTIFICATE (1 OR...,MARINE INSURANCE POLICY,46A,6281MLC00000321
3,4. PACKING LIST IN 3 ORIGINALS AND 2 COPIES.,PACKING LIST,46A,6281MLC00000321
4,5. CERTIFICATE OF ORIGIN ISSUED BY CHAMBER OF ...,CERTIFICATE OF ORIGIN,46A,6281MLC00000321
...,...,...,...,...
33390,2. DRAFT DRAWN UNDER THIS CREDIT HAS TO BEAR T...,not find,47A,0505621IM0000131
33391,"3. UNLESS OTHERWISE EXPRESSLY STATED, THIS CRE...",not find,47A,0505621IM0000131
33392,4. PRESENTATION TO BE MADE WITHIN THE VALIDITY...,not find,47A,0505621IM0000131
33393,5. IN ACCORDANCE WITH THE PROVISIONS OF ART 16...,not find,47A,0505621IM0000131


In [33]:
result['class'] = result['predict'].map(p2c)

In [34]:
result.head(20)

,string_X_train,predict,from,LCNO,class
0,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,COMMERCIAL INVOICE,46A,6281MLC00000321,商業發票
1,2. FULL SET OF SIGNED 'CLEAN ON BOARD' OCEAN B...,CERTIFICATE FROM SHIPPING COMPANY,46A,6281MLC00000321,船證
2,3.MARINE INSURANCE POLICY OR CERTIFICATE (1 OR...,MARINE INSURANCE POLICY,46A,6281MLC00000321,保險單據
3,4. PACKING LIST IN 3 ORIGINALS AND 2 COPIES.,PACKING LIST,46A,6281MLC00000321,裝箱單
4,5. CERTIFICATE OF ORIGIN ISSUED BY CHAMBER OF ...,CERTIFICATE OF ORIGIN,46A,6281MLC00000321,產地(商會)證明
5,6. CERTIFICATE ISSUED BY THE CARRIER OR ITS AG...,not find,46A,6281MLC00000321,NaN
6,7. CERTIFICATE OF ANALYSIS IN 2 ORIGINALS AND ...,CERTIFICATE OF ANALYSIS,46A,6281MLC00000321,檢驗證明
7,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,COMMERCIAL INVOICE,46A,L072875,商業發票
8,2. FULL SET CLEAN ON BOARD BILLS OF LADING MAD...,B/L,46A,L072875,提單
9,3. DETNORSK VERITAS.,not find,46A,L072875,NaN


In [35]:
result.tail(20)

,string_X_train,predict,from,LCNO,class
33375,"+ FOR EACH SET OF DOCUMENTS, BENEFICIARY'S CER...",not find,47A,P02L21112I000040,NaN
33376,+ IF DISCREPANCIES ON DOCUMENTS CHARGES FOR EU...,not find,47A,P02L21112I000040,NaN
33377,+ AN EXTRA PHOTOCOPY OF ALL DOCUMENTS IS REQUI...,not find,47A,P02L21112I000040,NaN
33378,+ DOCUMENTS MUST BE SENT TO US PER DHL OR ANY ...,not find,47A,P02L21112I000040,NaN
33379,1. ALL DOCUMENTS TO BE FORWARDED IN ONE COVER....,not find,47A,LC0956521000065,NaN
33380,2. PLEASE PROVIDE ONE EXTRA COPY OF COMMERCIAL...,COMMERCIAL INVOICE,47A,LC0956521000065,商業發票
33381,3. A DISCREPANCY FEE EQUAL TO USD60.00 WILL BE...,not find,47A,LC0956521000065,NaN
33382,4. AMOUNT OF EACH DRAWING MUST BE ENDORSED BY ...,not find,47A,LC0956521000065,NaN
33383,"5. DRAFT(S), IF ANY, SHOWING CORRECTION(S) OR ...",not find,47A,LC0956521000065,NaN
33384,7.\r\n.\r\n2. PLEASE PROVIDE ONE EXTRA COPY OF...,COMMERCIAL INVOICE,47A,LC0956521000065,商業發票


In [36]:
result[result['predict']!='not find'].shape

(16353, 5)

In [37]:
result[result['predict']=='not find'].shape

(17042, 5)

In [38]:
result.to_csv('../data/preprocess_for_SQUAD_押匯文件.csv')